In [2]:
import numpy as np
import torch
import torch.nn as nn
import pickle

from torch.autograd import Variable
from pdf_mog import *
from util import train_sgd

%matplotlib inline

In [3]:
example = pickle.load(open('example_1.pkl', 'rb'))

In [4]:
example

{'comment': 'posterior is (uncorrected * prior) / proposal',
 'posterior': <delfi.distribution.mixture.GaussianMixture.MoG at 0x7f62815157b8>,
 'prior': <delfi.distribution.Gaussian.Gaussian at 0x7f6242050e48>,
 'proposal': <delfi.distribution.Gaussian.Gaussian at 0x7f6242050ef0>,
 'uncorrected': <delfi.distribution.mixture.GaussianMixture.MoG at 0x7f6242058198>}